In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
from bench_util import *


emb_name = "treeMixed_150w_combined_Strat1"

word_dict,id_dict, matrix_normalized = load_embedding(".",emb_name)


dimensions 500
243604


In [2]:
def find_candidate(sim,exclusion_list_ids,isMult):
    for exclude_id in exclusion_list_ids:
        if isMult:
            sim[exclude_id] = 0#lowest possible value in 3CosMult
        else:
            sim[exclude_id] = -1#lowest possible value in 3CosAdd
    return np.argmax(sim)

In [3]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list

def unzip(some_list):
    return [ i for i, j in some_list ]
    
# take nearest 6 and remove the searched word itself
#print(unzip(find_nearest_k('bank' ,7)[1:]))#the food or the place
#print(unzip(find_nearest_k('apple',7)[1:]))#location or to to speak to
#print(unzip(find_nearest_k('so'   ,7)[1:]))#noun (animal) or verb 
#print(unzip(find_nearest_k('desert',   6)[1:]))#animal or verb
#print(unzip(find_nearest_k('left',   6)[1:]))#adverb (direction) and verb(the plane left)
#print(unzip(find_nearest_k('duck',   6)[1:]))# The president of the bank walked along the river bank.
#print("--------")

# 3cosAdd normalised befor arithmetic

In [4]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],False)
    return word_dict[index]

## 3cos Mult

In [5]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],True)
    return word_dict[index]

# Test all

In [6]:
def format_results(add,mult):
    add_string = str(add)
    add_string = add_string.lstrip('0')
    add_string = format(add_string, ".4")
    mult_string = str(mult)
    mult_string = mult_string.lstrip('0')
    mult_string = format(mult_string, ".4")
    return add_string +'/'+mult_string


In [7]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0]
count_syn_sucess      = [0,0]

errors = []


files = os.listdir('D:\\datasets\\question-data')
for idx,file_name in enumerate(sorted(files)):
    print(file_name)
    if file_name.endswith('.classes'):
        continue
    
    with open('D:\\datasets\\question-data\\'+file_name,'r') as file:    
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
         
        for line_id,line in enumerate(lines):
            
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_sem_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_syn_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
        print(format_results(local_sucesses[0]/float(local_tasks_count),local_sucesses[1]/float(local_tasks_count)))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[0]/float(count_sem_questions),count_sem_sucess[1]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[0]/float(count_syn_questions),count_syn_sucess[1]/float(count_syn_questions)))

print('overall')
print(format_results((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_syn_questions), \
                     (count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_syn_questions)))

capital-common-countries.txt
.804/.0
-------------------
capital-world.txt
.711/.0
-------------------
city-in-state.txt
.480/.0
-------------------
currency.txt
.006/.0
-------------------
family.txt
.434/.0
-------------------
gram1-adjective-to-adverb.txt
.109/.0
-------------------
gram2-opposite.txt
.086/.0
-------------------
gram3-comparative.txt
.247/.0
-------------------
gram4-superlative.txt
.055/.0
-------------------
gram5-present-participle.txt
.341/.0
-------------------
gram6-nationality-adjective.txt
.682/.0
-------------------
gram7-past-tense.txt
.444/.0
-------------------
gram8-plural.txt
.436/.0
-------------------
gram9-plural-verbs.txt
.271/.0
-------------------
ner.classes
treebank_pos.classes
universal_pos.classes
semantical
.568/.0
syntactical
.331/.0
overall
.438/.0


In [8]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [9]:
with open(r'D:\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
13256
16277
tf.Tensor(0.9819115283029549, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
13256
13256
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
156683
22286
tf.Tensor(0.9859224146624103, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
48235
22286
tf.Tensor(0.9925199562964687, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
55526
1685
tf.Tensor(0.9947516988056078, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
44865
1685
tf.Tensor(0.992981791782555, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
176672
117964
tf.Tensor(0.5121265281750983, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
28945
23501
tf.Tensor(0.1953110536010459, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
118061
44767
tf.Tensor(0.9829688031276992, shape=(), dtype=float64) 0.7
lineprofessor	doctor	6.62

prof

tf.Tensor(0.974186999455, shape=(), dtype=float64) 0.747
lineseafood	food	8.34

seafood food
113842
96769
tf.Tensor(0.971412667981168, shape=(), dtype=float64) 0.834
lineseafood	lobster	8.70

seafood lobster
113842
212405
tf.Tensor(0.9583634469705204, shape=(), dtype=float64) 0.8699999999999999
linelobster	food	7.81

lobster food
212405
96769
tf.Tensor(0.9582901603551537, shape=(), dtype=float64) 0.7809999999999999
linelobster	wine	5.70

lobster wine
212405
98442
tf.Tensor(0.9565906221463347, shape=(), dtype=float64) 0.5700000000000001
linechampionship	tournament	8.36

championship tournament
88164
52037
tf.Tensor(0.9931230194711682, shape=(), dtype=float64) 0.836
lineman	woman	8.30

man woman
67022
76513
tf.Tensor(0.9961515246489626, shape=(), dtype=float64) 0.8300000000000001
lineman	governor	5.25

man governor
67022
76114
tf.Tensor(0.9887038813458494, shape=(), dtype=float64) 0.525
linemurder	manslaughter	8.53

murder manslaughter
15784
169403
tf.Tensor(0.937292719817872, shape=(), 

tf.Tensor(0.9828825228562814, shape=(), dtype=float64) 0.3
lineprejudice	recognition	3.00

prejudice recognition
85215
136319
tf.Tensor(0.9643319974159519, shape=(), dtype=float64) 0.3
lineviewer	serial	2.97

viewer serial
60787
144040
tf.Tensor(0.9655788367625248, shape=(), dtype=float64) 0.29700000000000004
linepeace	insurance	2.94

peace insurance
41729
29008
tf.Tensor(0.9803606827232012, shape=(), dtype=float64) 0.294
lineMars	water	2.94

Mars water
0 0.294
linemedia	gain	2.88

media gain
44865
81863
tf.Tensor(0.988447389766103, shape=(), dtype=float64) 0.288
lineprecedent	cognition	2.81

precedent cognition
91951
123635
tf.Tensor(0.39171092954910036, shape=(), dtype=float64) 0.281
lineannouncement	effort	2.75

announcement effort
84433
116376
tf.Tensor(0.982388810789452, shape=(), dtype=float64) 0.275
lineline	insurance	2.69

line insurance
33080
29008
tf.Tensor(0.9848695108859079, shape=(), dtype=float64) 0.269
linecrane	implement	2.69

crane implement
110138
4876
tf.Tensor(0.974

In [10]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.18109410485955738, pvalue=0.009718842141067875)